In [50]:
import pandas as pd
import os

os.chdir("C:/Users/jstelman/Git/chelsvig_urban_pesticides/src/")
from path_names import *

obs_path = os.path.join(main_path, "observed_data", "SURF_water_placer_bifenthrin.csv")
print(obs_path)

outfalls = ['outfall_31_26', 'outfall_31_28', 'outfall_31_29', 'outfall_31_35', 
            'outfall_31_36', 'outfall_31_38', 'outfall_31_42']

def output_path(of, i):
    op_path = os.path.join(vvwm_path, of, ("input_" + str(i)), "output_NPlesant_Custom_parent_daily.csv")
    return(op_path)

C:\Users\jstelman\Git\chelsvig_urban_pesticides\observed_data\SURF_water_placer_bifenthrin.csv


In [51]:
obs_data = pd.read_csv(obs_path, usecols=["Concentration..ppb.","Level_of_quantification..ppb.",
                                         "Method_detection_level..ppb.", "Sample_date", "Site_code"],
                      parse_dates=["Sample_date"])

for idx, row in obs_data[obs_data['Concentration..ppb.']==0].iterrows():
    obs_data.loc[idx,'Concentration..ppb.'] = min([row['Level_of_quantification..ppb.'],
                                                   row['Method_detection_level..ppb.']])/2

obs_data = obs_data.drop(labels = ["Level_of_quantification..ppb.","Method_detection_level..ppb."], axis = 1)
obs_data.head()

,Concentration..ppb.,Sample_date,Site_code
0,0.00235,2015-06-08,31_26
1,0.00235,2010-06-24,31_44
2,0.00235,2009-05-06,31_44
3,0.00235,2010-09-03,31_44
4,0.00035,2016-10-14,31_26


In [52]:
sim1_of26_data = pd.read_csv(output_path("outfall_31_26", 1), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
sim1_of26_data['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
sim1_of26_data['Site_code'] = '31_26'
sim1_of26_data.head()

,davg_bif_conc,Sample_date,Site_code
0,0.0,2009-01-01,31_26
1,0.0,2009-01-02,31_26
2,0.0,2009-01-03,31_26
3,0.0,2009-01-04,31_26
4,0.0,2009-01-05,31_26


In [53]:
comp_data_ip1 = sim1_of26_data.merge(obs_data, how = "inner", on = ['Sample_date','Site_code'])

In [54]:
for of in outfalls[1:]:
    dat = pd.read_csv(output_path(of, 1), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
    dat['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
    dat['Site_code'] = of[-5:]
    comp_data_ip1 = comp_data_ip1.append(dat.merge(obs_data, how = "inner", on = ['Sample_date','Site_code']), ignore_index=True)

In [55]:
comp_data_ip1

,davg_bif_conc,Sample_date,Site_code,Concentration..ppb.
0,1.744700e+07,2009-02-13,31_26,0.017000
1,8.595200e+07,2009-04-08,31_26,0.007740
2,7.793400e+09,2009-04-13,31_26,0.000880
3,1.738700e+11,2009-05-01,31_26,0.008800
4,1.026300e+10,2009-08-28,31_26,0.000880
...,...,...,...,...
101,1.385500e+10,2010-07-19,31_42,0.000880
102,5.664500e+07,2010-10-24,31_42,0.000880
103,7.886300e+10,2011-02-16,31_42,0.000880
104,6.527900e+10,2011-05-15,31_42,0.000455


In [56]:
import hydroeval

In [68]:
NSE1 = 1 - ((sum(comp_data_ip1['davg_bif_conc'] - comp_data_ip1['Concentration..ppb.'])**2)/
            (sum(comp_data_ip1['Concentration..ppb.'] - 
                 sum(comp_data_ip1['Concentration..ppb.'])/len(comp_data_ip1['Concentration..ppb.']))**2))

In [69]:
NSE1

-4.060104162103298e+55